In [1]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, Output, interact_manual
import textwrap
from screeninfo import get_monitors

In [ ]:
print("Enter contrasting legal statuses")
heading_1 = input("Status 1 (e.g valid, not prohibited, permissible):")
heading_2 = input("Status 2 (e.g invalid, prohibited, inpermissible):")

In [ ]:
print("Enter act to be decided")
act = input("Act: ")

In [ ]:
print("Are you on windows/pc or mac? ")
check = True
while check:
        os_type = input("Enter 1 for windows/pc and 2 for mac:")
        if os_type in ['1', '2']:
            check = False
        else:
            print("Invalid input. Please enter 1 or 2.")
            
print("Where do you want plots saved? For M. Berman, use default input:")
print("Default: C:\\Users\\mitchber\\OneDrive - University of Pennsylvania Law School\\02-The Book_TSWTLI\\Diagrams")
total_path = input("Path name:")

In [6]:
#Need to figure out screeninfo install for this to work
def get_screen_dimensions():
    monitor = get_monitors()[0]
    width = monitor.width
    height = monitor.height
    return width, height

def set_dynamic_figsize():
    screen_width, screen_height = get_screen_dimensions()
    base_width, base_height = 10, 6
    scaling_factor = 0.5
    fig_width = base_width * scaling_factor * (screen_width / 1920)
    fig_height = base_height * scaling_factor * (screen_height / 1080)
    return (fig_width, fig_height)

In [ ]:
principle_widgets = []
principles_for = []
principles_against = []
default_values = {
    'length': 50,
    'thickness': 1.0
}

# Function to create widgets for each principle
def create_principle_widgets():
    global principle_widgets
    principle_widgets = []

    for label in principles_for:
        length_widget = widgets.IntSlider(
            min=1, max=100, step=1, value=default_values['length'],
            description=label,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='600px')
        )
        thickness_widget = widgets.FloatSlider(
            min=0.1, max=5, step=.1, value=default_values['thickness'],
            description=label,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='600px')
        )
        principle_widgets.append((length_widget, thickness_widget, label, 1))

    for label in principles_against:
        length_widget = widgets.IntSlider(
            min=1, max=100, step=1, value=default_values['length'],
            description=label,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='600px')
        )
        thickness_widget = widgets.FloatSlider(
            min=0.1, max=5, step=0.1, value=default_values['thickness'],
            description=label,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='600px')
        )
        principle_widgets.append((length_widget, thickness_widget, label, -1))

# Function to plot principles based on current slider values
def plot_principles(save_path=None):
    with output:
        output.clear_output(wait=True)
        #fig, ax = plt.subplots(figsize = set_dynamic_figsize())
        fig, ax = plt.subplots()

        plt.rcParams['font.family'] = 'serif'
        plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

        num_is = len(principles_for)
        num_not = len(principles_against)
        max_num = max(num_is, num_not)

        if max_num >= 2:
            #x_lim = (max_num)/2 + 0.25
            #y_lim = max_num + 0.5
            lim = max_num + 0.5
        else:
            #x_lim = 2
            #y_lim = 2
            lim = 1.5

        #ax.set_xlim(-x_lim, x_lim)
        #ax.set_ylim(0, y_lim)
        ax.set_xlim(0, lim)
        ax.set_ylim(0,lim)

        #y_offset_is = y_lim/(num_is+1)
        #y_offset_not = y_lim/(num_not+1)
        y_offset_is = lim/(num_is+1)
        y_offset_not = lim/(num_not+1)
        sum_for = 0
        sum_against = 0

        for length_widget, thickness_widget, label, direction in principle_widgets:
            # length = x_lim * 0.35 * (1+length_widget.value/100)
            # thickness = y_lim * (1+thickness_widget.value/100)
            # sum_length = length_widget.value
            # sum_thickness = thickness_widget.value
            # x_start = -x_lim+(x_lim*.05) if direction == 1 else x_lim-(x_lim*0.05)
            # x_end = x_start + direction * length
            length = lim/2 * 0.9 * (length_widget.value/100)
            thickness = 5*(thickness_widget.value) #Change first value to make standard width bigger
            sum_length = length_widget.value
            sum_thickness = thickness_widget.value
            #x_start = 0 +(x_lim*.05) if direction == 1 else x_lim-(x_lim*0.05)
            x_start = 0 +(lim *.05) if direction == 1 else lim-(lim*0.05)
            x_end = x_start + direction * length

            arrow_style = f"Simple,tail_width={0.5},head_width={1.1},head_length={1.25}"

            if direction == 1:
                arrow = patches.FancyArrowPatch((x_start, y_offset_is), (x_end, y_offset_is),
                                                connectionstyle="arc3",
                                                arrowstyle=arrow_style,
                                                color='lightblue',
                                                mutation_scale=thickness,
                                                linewidth=thickness)
                ax.add_patch(arrow)
                ax.text((x_start + x_end) / 2, y_offset_is-0.02, label, fontsize=10, ha='center')
                #y_offset_is += y_lim/(num_is+1)
                y_offset_is += lim/(num_is+1)
                sum_for += (sum_length/100)*(sum_thickness)

            else:
                arrow = patches.FancyArrowPatch((x_start, y_offset_not), (x_end, y_offset_not),
                                                connectionstyle="arc3",
                                                arrowstyle=arrow_style,
                                                color='orange',
                                                mutation_scale=thickness,
                                                linewidth=thickness)
                ax.add_patch(arrow)
                ax.text((x_start + x_end) / 2, y_offset_not-0.02, label, fontsize=10, ha='center')
                #y_offset_not += y_lim/(num_not+1)
                y_offset_not += lim/(num_not+1)
                sum_against += (sum_length/100)*(sum_thickness)

        #Heading creation
        # ax.text(- 0.5, y_lim + .1, heading_1, fontsize=20 + max_num, ha='center', va='center', color='blue')
        # ax.text(0.5, y_lim + .1, heading_2, fontsize=20 + max_num, ha='center', va='center', color='red')
        ax.text(lim/2- lim*0.25, lim + lim*0.05, heading_1, fontsize=18, ha='center', va='center', color='orange')
        ax.text(lim/2+lim*0.25, lim + lim*0.05, heading_2, fontsize=18, ha='center', va='center', color='lightblue')
        ax.axvline(x=lim/2, color='gray', linestyle='--')

        circle_position = (sum_for - sum_against)*(lim/50) + lim/2 #Change the lim/50 statement to make circle move more or less
        #radius = 0.2*y_lim
        radius = 0.1*lim #Reduce the number to shrink the circle
        #circle = patches.Circle((circle_position, y_lim/2), radius=radius, color='green', alpha=0.5)
        circle = patches.Circle((circle_position, lim/2), radius=radius, color='green', alpha=0.5)
        ax.add_patch(circle)

        #ax.text(circle_position, y_lim / 2, act, fontsize=20, ha='center', va='center', color='black')
        ax.text(circle_position, lim / 2, act, fontsize=10, ha='center', va='center', color='black')


        ax.grid(False)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        ax.set_aspect('equal')

        plt.tight_layout()

        if save_path:
            #print(f"Saving path at {save_path}")
            fig.savefig(save_path, bbox_inches='tight')

        plt.show()

# Function to save the plot
def save_plot():
    path_name = act.replace(" ", "")
    if os_type == 1:
        plot_principles(save_path=f'{total_path}\{path_name}_plot.png')
    else:
        plot_principles(save_path=f'{total_path}/{path_name}_plot.png')

# Creating text input widgets
style = {'description_width': 'initial'}

text_input_for = widgets.Text(
    value='',
    placeholder='Enter principle',
    description=f'Principle supporting {heading_2}:',
    disabled=False,
    style=style
)

text_input_against = widgets.Text(
    value='',
    placeholder='Enter principle',
    description=f'Principle supporting {heading_1}:',
    disabled=False,
    style=style
)

# Function to update principles and plot
def update_plot(principle_for, principle_against):
    if principle_for:
        principles_for.append(principle_for)
        text_input_for.value = ''
    if principle_against:
        principles_against.append(principle_against)
        text_input_against.value = ''

    #Making sure functions are not called when the text boxes are cleared
    if principle_for or principle_against:
        create_principle_widgets()
        plot_principles()
        display_widgets()
    

# Using interact_manual for better control with a custom button label
im = interact_manual.options(manual_name = "Add principles")
im(update_plot, principle_for=text_input_for, principle_against=text_input_against);
#interact_manual(update_plot, principle_for=text_input_for, principle_against=text_input_against);


# Creating the output widget to display the plot
output = Output()
display(output)

# Save button
save_button = widgets.Button(description='Save Plot')
save_button.on_click(lambda x: save_plot())
display(save_button)

def display_widgets():
    length_widgets = widgets.VBox([
        widgets.Label(value='Percentage of Activation', layout=widgets.Layout(font_weight='bold')),
        *[widgets.HBox([length_widget]) for length_widget, _, _, _ in principle_widgets]
    ])

    thickness_widgets = widgets.VBox([
        widgets.Label(value='Relative Importance', layout=widgets.Layout(font_weight='bold')),
        *[widgets.HBox([thickness_widget]) for _, thickness_widget, _, _ in principle_widgets]
    ])

    widgets_box = widgets.HBox([length_widgets, thickness_widgets])
    display(widgets_box)

    # Re-attaching observers to all sliders
    for length_widget, thickness_widget, _, _ in principle_widgets:
        length_widget.observe(on_value_change, names='value')
        thickness_widget.observe(on_value_change, names='value')

def on_value_change(change):
    plot_principles()

plot_principles()